In [2]:
import customtkinter as ctk
import pandas as pd
import numpy as np
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt')
import warnings
warnings.filterwarnings("ignore")

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\crick\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\crick\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# Function to preprocess the text
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalnum()]
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return " ".join(tokens)

# Load and preprocess data
reviews_df = pd.read_csv("user_reviews.csv")

# Clean and preprocess data
reviews_df.dropna(subset=['Translated_Review', 'Sentiment'], inplace=True)
reviews_df = reviews_df[['Translated_Review', 'Sentiment']]
reviews_df = reviews_df[reviews_df['Sentiment'].isin(['Positive', 'Negative'])]
reviews_df['Sentiment_Encoded'] = reviews_df['Sentiment'].map({'Positive': 1, 'Negative': 0})
reviews_df['Processed_Review'] = reviews_df['Translated_Review'].apply(preprocess_text)

# TF-IDF Vectorizer
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(reviews_df['Processed_Review'])
y = reviews_df['Sentiment_Encoded']

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train models
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

lr_classifier = LogisticRegression(max_iter=200, random_state=42)
lr_classifier.fit(X_train, y_train)

svc_classifier = SVC(kernel="linear", random_state=42)
svc_classifier.fit(X_train, y_train)

# Predict sentiment for user input
def predict_sentiment(input_text, model):
    processed_text = preprocess_text(input_text)
    text_tfidf = tfidf.transform([processed_text])
    prediction = model.predict(text_tfidf)
    sentiment_label = "Positive" if prediction[0] == 1 else "Negative"
    return sentiment_label

In [4]:
# GUI setup using customtkinter
def create_gui():
    # Create the main window
    window = ctk.CTk()
    window.title("Sentiment Prediction")
    window.geometry("600x400")
    
    # Create a frame for the widgets
    frame = ctk.CTkFrame(window)
    frame.pack(pady=20, padx=20, fill="both", expand=True)
    
    # Title label
    title_label = ctk.CTkLabel(frame, text="Sentiment Analysis", font=("Helvetica", 24))
    title_label.pack(pady=10)
    
    # Text input field for the review
    review_label = ctk.CTkLabel(frame, text="Enter your review:", font=("Helvetica", 16))
    review_label.pack(pady=10)
    review_input = ctk.CTkEntry(frame, font=("Helvetica", 16), width=400)
    review_input.pack(pady=10)
    
    # Function to display predictions
    def display_predictions():
        user_input = review_input.get()
        if user_input:
            rf_sentiment = predict_sentiment(user_input, rf_classifier)
            lr_sentiment = predict_sentiment(user_input, lr_classifier)
            svc_sentiment = predict_sentiment(user_input, svc_classifier)

            # Display sentiment predictions
            rf_label.configure(text=f"Random Forest: {rf_sentiment}")
            lr_label.configure(text=f"Logistic Regression: {lr_sentiment}")
            svc_label.configure(text=f"Support Vector Classifier: {svc_sentiment}")
        else:
            result_label.config(text="Please enter a review!", text_color="red")
    
    # Button to trigger prediction
    predict_button = ctk.CTkButton(frame, text="Predict Sentiment", font=("Helvetica", 16), command=display_predictions)
    predict_button.pack(pady=20)
    
    # Display labels for predictions
    result_label = ctk.CTkLabel(frame, text="Prediction Results:", font=("Helvetica", 18))
    result_label.pack(pady=10)
    
    rf_label = ctk.CTkLabel(frame, text="Random Forest: ", font=("Helvetica", 16))
    rf_label.pack(pady=5)
    
    lr_label = ctk.CTkLabel(frame, text="Logistic Regression: ", font=("Helvetica", 16))
    lr_label.pack(pady=5)
    
    svc_label = ctk.CTkLabel(frame, text="Support Vector Classifier: ", font=("Helvetica", 16))
    svc_label.pack(pady=5)
    
    # Start the GUI main loop
    window.mainloop()

# Run the GUI application
create_gui()
